In [1]:
!pip install extra_keras_datasets -q

In [2]:
from matplotlib import pyplot as plt
import numpy as np
from numpy.linalg import matrix_power
import random
import math as m
import scipy

In [3]:
def RobustGC(W, y_train_prime, eta):
  D = np.zeros ((len(W),len(W)))
  for i in range (len(W)):
    D[i][i]= np.sum(W[i] , axis=0)

  D_inv = np.sqrt(matrix_power(D,-1))
  S = D_inv @ W @ D_inv
  LN = np.eye(len(S)) - S
  V0 = np.zeros(len(D))

  for i in range (len (D)):
    V0[i]= np.sqrt(D[i][i])

  V0_norm = np.linalg.norm(V0, 2)
  V0= V0/V0_norm
  v=np.linalg.eigvals(LN)
  v=sorted(v)
  lambda_1 = v[1]
  gamma_t= eta*lambda_1
  # print(gamma_t)
  
  f = np.linalg.inv((LN/gamma_t)-np.eye(len(LN)))@(y_train_prime - (np.dot(V0,np.dot(V0.transpose(), y_train_prime))))
  pred= np.sign(f)
  pred = np.squeeze(np.asarray(pred))
  # print (pred)

  return pred, gamma_t

In [4]:
def PF_RobustGC(W, y_train_prime):
  D = np.zeros ((len(W),len(W)))
  for i in range (len(W)):
    D[i][i]= np.sum(W[i] , axis=0)

  D_inv = np.sqrt(matrix_power(D,-1))
  S = D_inv @ W @ D_inv
  LN = np.eye(len(S)) - S
  V0 = np.zeros(len(D))

  for i in range (len (D)):
    V0[i]= np.sqrt(D[i][i])

  V0_norm = np.linalg.norm(V0, 2)
  V0= V0/V0_norm
  v=np.linalg.eigvals(LN)
  v=sorted(v)
  lambda_1 = v[1]
  gamma_t= 0.9*lambda_1
  # print(gamma_t)
  
  f = np.linalg.inv((LN/gamma_t)-np.eye(len(LN)))@(y_train_prime - (np.dot(V0,np.dot(V0.transpose(), y_train_prime))))
  pred= np.sign(f)
  pred = np.squeeze(np.asarray(pred))
  # print (pred)

  return pred, gamma_t

In [5]:
def accuracy (y_pred , y_orig):
  count=0
  for i in range (len(y_pred)):
    if y_pred[i]==y_orig[i]:
      count+=1
  result = count/len(y_pred) * 100
  return result

In [25]:
from extra_keras_datasets import usps
(X_train, y_train), (X_test,y_test) = usps.load_data()
filter_nine = np.where(y_train == 9)
filter_four = np.where(y_train == 4)
X_train_9 = X_train[filter_nine]
X_train_4 = X_train[filter_four]
X_train_9_125 = X_train_9[0:125]
X_train_4_125 = X_train_4[0:125]
y_train = [1]*125 + [-1]*125
X_train = np.append(X_train_4_125, X_train_9_125, axis=0)
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_train = X_train.reshape(-1, 16*16)
def GaussianKernel(v1, v2, sigma):
  #sigma = 0.47
  dist = scipy.spatial.distance.braycurtis(v1, v2)
  return scipy.exp(-np.sum(dist**2) / (2.*sigma**2))

W = np.zeros((len(X_train), len(X_train)))
#W=polynomial_kernel(X_train, gamma=26.5 ,degree=3 )
for i in range(len(X_train)):
  for j in range(len(X_train)):
    W[i][j] = GaussianKernel(X_train[i], X_train[j], sigma=0.47)
acc = np.zeros (20)
for i in range (20):
  y_train_prime = y_train.copy()
  ra = random.sample(range(0, 250), 125)
  for j in ra:
    y_train_prime[j]=0  
  pred , gamma_t = RobustGC (W, y_train_prime , 0.85)
  acc[i] = accuracy (pred , y_train)
print (np.mean(acc))

INFO:root:Loading dataset = usps
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: scipy.exp is deprecated and will be removed in SciPy 2.0.0, use numpy.exp instead


93.97999999999999


In [7]:
from extra_keras_datasets import usps
(X_train, y_train), (X_test,y_test) = usps.load_data()
filter_nine = np.where(y_train == 9)
filter_four = np.where(y_train == 4)
X_train_9 = X_train[filter_nine]
X_train_4 = X_train[filter_four]
X_train_9_125 = X_train_9[0:125]
X_train_4_125 = X_train_4[0:125]
y_train = [1]*125 + [-1]*125
X_train = np.append(X_train_4_125, X_train_9_125, axis=0)
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_train = X_train.reshape(-1, 16*16)
def GaussianKernel(v1, v2, sigma):
  #sigma = 0.47
  dist = scipy.spatial.distance.braycurtis(v1, v2)
  return scipy.exp(-np.sum(dist**2) / (2.*sigma**2))

W = np.zeros((len(X_train), len(X_train)))
#W=polynomial_kernel(X_train, gamma=26.5 ,degree=3 )
for i in range(len(X_train)):
  for j in range(len(X_train)):
    W[i][j] = GaussianKernel(X_train[i], X_train[j], sigma=0.47)
acc = np.zeros (20)
for i in range (20):
  y_train_prime = y_train.copy()
  ra = random.sample(range(0, 250), 125)
  for j in ra:
    y_train_prime[j]=0  
  pred , gamma_t = PF_RobustGC (W, y_train_prime)
  acc[i] = accuracy (pred , y_train)
print (np.mean(acc))

INFO:root:Loading dataset = usps
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: scipy.exp is deprecated and will be removed in SciPy 2.0.0, use numpy.exp instead


94.03999999999999
